In [1]:
import numpy as np
np.random.seed(42)
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import missingno as msno
import keras
import keras.backend as K
from keras.layers import *
from keras.models import *
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report 
from sklearn.svm import SVC, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from keras.activations import *
import math
from itertools import product
from sklearn.model_selection import RandomizedSearchCV
from functools import partial
import pickle
import time
from IPython.core.magic import (register_line_magic,
                                register_cell_magic)
%matplotlib inline

Using TensorFlow backend.


In [2]:
address_corr = "corr_atf/"
if not os.path.exists(address_corr):
    os.mkdir(address_corr)
    print("path created")
else :
    print("path was available")


path created


In [23]:
def get_global_vars(first=False, offset = 0):
    global rfs
    global df1
    global df 
    global ommit
    global otherCpks
    global cols
    global num
    global cret_index
    global gender_index
    global age_index
    
    if first : 
        
        df1 = pd.read_csv("fscore.csv")
        
    rfs = []
    ommit = ['creati_1', 'sgot_1', 'phosph_1']
    otherCpks = ['lcpk', 'cpkf', 'cpkt', 'cpkg',]
    cols = ['cpk_1', 'ldh_1', 'wbc_1','plt_1', 'uricac_1',
            'na_1', 'k_1', 'ca_1',
            'age', 'gender','crat3f', "atn"]
    cols = ommit + cols
    df = df1[cols]
    num = len(cols) - len([ 'age', 'gender','crat3f', "atn" ]) + offset
    cret_index = cols.index('crat3f')
    gender_index = cols.index('gender')
    age_index = cols.index('age')
    print(df.columns)

In [4]:
def getGender(x):
    '''
    2 equals female
    other equals to male
    '''
    if x == 2 :
        return 0
    elif x == 1 : 
        return 1
    elif x == 3 :
        return 1
    else:
        return np.nan

In [5]:
def replaceNan(mat_in, num_in, cols, means, gender):
    
    '''
    replace the columns that we know of with the average found with in human body
    '''
    
    res = mat_in.copy()
    
    for i, iv in zip(itertools.count(), means):
        if iv not in cols :
#             because we may remove things
            continue
        i = list(cols).index(iv)
                
        for j, jv in enumerate(mat_in[ : , i]):
            
            mv = means[iv]
            
            if str(jv) == str(np.nan):
                
                if type(mv) == dict :
                    
                    
                    if gender_index is -1 :
                        mv = (mv["male"] + mv["female"]) / 2.0
                    
                    elif mat_in[j, gender] == 1 :
                        
                        mv = mv["male"]
                    
                    elif mat_in[j, gender] == 0 :
                        
                        mv = mv["female"]
                    
                    else:
                        '''
                        problem because we have 3 too!!
                        we handled it by checking
                        '''
                        mv = (mv["male"] + mv["female"]) / 2.0
                        
                res[j, i] = mv
        
    return res

In [6]:
def get_model_for_age(shape, lr = 1e-3):
    
    
    '''
    tries to predict the continues value of age
    '''
    
    in1 = Input(shape)
    
    X   = Dense(20, activation=relu)(in1)
    X   = Dropout(0.5)(X)
#     X   = Dense(10, activation=relu)(X)
#     X   = Dropout(0.6)(X)
#     X   = Dense(5, activation=relu)(X)
    
    X   = Dense(1, activation=linear)(X)
    
    
    model = Model(in1, X)
    
    np.random.seed(42)
    
    model.compile(keras.optimizers.Adam(lr), keras.losses.mse)
    
    return model


In [7]:
def get_model_cret(shape, lr = 1e-3):
    
    '''
    tries to predict the continues value of cret
    '''
    
    in1 = Input(shape)
    
#     X = Dropout(0.0)(in1)
    X = Dense(20, activation=relu)(in1)
#     X = Dense(10, activation=relu)(X)
# #     X = Dropout(0.4)(X)
#     X = Dense(10, activation=relu)(X)
#     X = Dense(5, activation=relu)(X)
    X = Dropout(0.3)(X)
#     X = Dense(5, activation=relu)(X)
#     X = Dense(2, activation=relu)(X)
    X = Dense(2, activation=relu)(X)
    X = Dense(1, activation=relu)(X)
    
    
    model = Model(in1, X)
    
    np.random.seed(42)
    
    model.compile(keras.optimizers.Adam(lr), keras.losses.mse)
    
    
    return model


In [8]:
def get_model_final(shape, lr = 1e-3):
    
    in1 = Input(shape)
    
    X = Dense(20, activation=relu)(in1)
#     X = BatchNormalization()(X)
#     X = Dense(10, activation=relu)(X)
#     X = BatchNormalization()(X)
#     X = Dense(10, activation=relu)(X)
#     X = BatchNormalization()(X)
# #     X = Dense(5, activation=relu)(X)
#     X = Dropout(0.4)(X)
#     X = Dense(5, activation=relu)(X)
    modelMiddle = Model(in1, X)
    X = Dropout(0.3)(X)
    X = Dense(1, activation=sigmoid)(X)
    
    
    model = Model(in1, X)
    
    np.random.seed(42)
    
    model.compile(keras.optimizers.Adam(lr), 
                        keras.losses.binary_crossentropy, metrics=["accuracy"])
    
    return model, modelMiddle


In [9]:
def get_model_final_avg(shape, lr = 1e-3):
    
    in1 = Input(shape)
    
    X = Dense(20, activation=relu)(in1)
#     X = BatchNormalization()(X)
#     X = Dense(10, activation=relu)(X)
#     X = Dense(10, activation=relu)(X)
#     X = BatchNormalization()(X)
#     X = Dense(5, activation=relu)(X)
#     X = BatchNormalization()(X)
#     X = Dense(5, activation=relu)(X)
    modelMiddle = Model(in1, X)
    X = Dense(1, activation=sigmoid)(X)
    
    
    model = Model(in1, X)
    
    np.random.seed(42)
    
    model.compile(keras.optimizers.Adam(lr), 
                        keras.losses.binary_crossentropy, metrics=["accuracy"])
    
    
    return model, modelMiddle


In [10]:
def train_model(model, type_model,X, y, epochs=None, batch_size=None, path_to_save=None):
    '''
    trains the model with data given to it , used for being dynamic
    '''
    
#     print(type_model)
    if type_model == "nn":
        model.fit(X, y, 
                  epochs=epochs, batch_size=batch_size, verbose = 0)
#         model.save(path_to_save)
    else:
        model.fit(X, y)

In [11]:
def get_model_train_and_replace(model, data_before, index, type_model,
                                path_to_save = None, normalize = True,p = 0.9,
                                epochs = 50, batch_size = 8, 
                                classification = True, normalize_final_data = True, train = True,
                               scaler = None, ifCret = False):
    '''
    for one coloumn , uses model to predict its values
    '''
    
    data = pd.DataFrame(data_before[:, : index + 1]).dropna().values
    if train :
        scaler = preprocessing.StandardScaler().fit(data)
    data = scaler.transform(data)
    
    
    
    X = data[:, : -1]
    y = data[:, -1 ]
    
    if train :
        train_model(model, type_model, X, y,epochs, batch_size, path_to_save)
#     so the outpout is normalized
    
    res = data_before.copy()
    
    it = 0
    if ifCret:
        if cret_index is not -1 :
            res[:, index] = np.array([np.nan for i in range(res.shape[0])])
    
#     print(res.shape[0], data_before.shape[0], "look at me")
    for i in range(res.shape[0]):
        if str(res[i, index]) == str(np.nan):
            if (classification) :
                res[i, index] = model.predict(res[i:i+1, : index]).reshape(-1)[0] > 0.5
            else :
                res[i, index] = model.predict(res[i:i+1, : index]).reshape(-1)[0]
        else :
            res[i, index] = data[it, index]
            it += 1
            
    
    return model, res, scaler
    
    
    

In [12]:
def replace_with_ml(data, model_age, model_cret, type_model, train = True,
                   scaler_age = None, scaler_cret = None):
   
    '''
    you give all the models and gives the data completed
    '''
    x = data
    if age_index is not -1 :
        model_age, age_achived, scaler_age = get_model_train_and_replace(
            model_age, data,
            age_index, type_model, "model_age_complete.h5", classification = False,
            epochs = 100, train = train, scaler = scaler_age
        )   
        x = age_achived
    else :
        model_age = None
        scaler_age = None
    if gender_index is not -1 :
        age_achived_df = pd.DataFrame(x, columns=cols)
        gender_achived_df = age_achived_df[age_achived_df["gender"].isnull()^1]
        gender_achived = gender_achived_df.values
        x = gender_achived
    else :
        gender_achived = None
    if cret_index is not -1 :
        model_cret, cret_achived, scaler_cret = get_model_train_and_replace(
            model_cret, x,
            cret_index, type_model, "model_cret_complete.h5", classification = False,
            epochs = 100, train = train, scaler = scaler_cret, ifCret = True
        )
        x = cret_achived
    else :
        model_cret = None
        scaler_cret = None
    
    return x, scaler_age, scaler_cret, model_age, model_cret

In [13]:
def get_models():
    '''
    gives you all the models
    '''
    if age_index == -1 :
        model_age_nn = None
    else:
        model_age_nn   = get_model_for_age([age_index])
    if cret_index == -1 :
        model_cret_nn = None
    else :
        model_cret_nn  = get_model_cret([cret_index])
    model_final_nn, mid_nn = get_model_final([len(cols) - 1], lr=1e-3)
    model_final_for_avg, mid_nn_avg = get_model_final_avg([len(cols) - 1], lr=5e-4)

    
    
    param_grid = {'C':[0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000,5000, 10000], "gamma":[1, 0.5,0.1, 0.05,0.01, 0.005, 0.001, 0.0005, 0.0001], 'kernel': ['rbf']}
    model_age_svr = GridSearchCV(SVR(), param_grid=param_grid, refit=True, verbose=0)
    model_cret_svr = GridSearchCV(SVR(), param_grid=param_grid, refit=True, verbose=0)
    model_final_svc = GridSearchCV(SVC(random_state=42), param_grid=param_grid, refit=True, verbose=0)
    
    
    
    n_estimators = [int(x) for x in np.linspace(start = 1, stop = 2000, num = 100)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, True]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'bootstrap': bootstrap}
    
    
    model_age_rfr = RandomForestRegressor(300)
    model_cret_rfr = RandomForestRegressor(300)
    model_final_rfc = RandomForestClassifier(random_state=42)
    
    rf_random = GridSearchCV(estimator = model_final_rfc, param_grid = random_grid, verbose=0, n_jobs = -1)
    return model_age_nn, model_cret_nn, model_final_nn, model_age_svr, model_cret_svr, model_final_svc, model_age_rfr, model_cret_rfr, rf_random, model_final_for_avg, mid_nn, mid_nn_avg


In [14]:
def get_acc(y_true, y_pred):
    
    '''
    gives rec0 and rec1
    '''
    
    arr = metrics.confusion_matrix(y_true, y_pred)
    rec0 = arr[0][0] / (arr[0][0]+arr[0][1])
    rec1 = arr[1][1] / (arr[1][1]+arr[1][0])
    return rec0, rec1

In [15]:
def get_res(*ins):
    '''
    this function will return the ensemble of the diffrent predictions
    it is made for binary classification (hence the 2)
    all the inputs have to be either 0 or 1
    '''
    
    res = np.zeros(shape=(len(ins[0]), 2))
    
    for i in ins :
        for j, jv in enumerate(i):
            res[j, int(jv)] += 1
    
    res = np.argmax(res, axis=1)
    
    for i, iv in enumerate(ins[-1].reshape(-1)):
        if iv == 1 :
            if ins[-2].reshape(-1)[i] == 1 :
                res[i] = 1
    
#     ones = ins[-1] == ins[-2]
#     ones = np.argwhere(ones)
#     np.put(res, np.ones_like(ones), ones)
    
    
    return res

In [16]:
def get_res(*ins):
    '''
    this function will return the ensemble of the diffrent predictions
    it is made for binary classification (hence the 2)
    all the inputs have to be either 0 or 1
    '''
    
    res = np.zeros(shape=(len(ins[0]), 2))
    
    for i in ins :
        for j, jv in enumerate(i):
            res[j, int(jv)] += 1
    
    res = np.argmax(res, axis=1)
    
    for i, iv in enumerate(ins[-1].reshape(-1)):
        if iv == 1 :
            if ins[-2].reshape(-1)[i] == 1 :
                res[i] = 1
    
#     ones = ins[-1] == ins[-2]
#     ones = np.argwhere(ones)
#     np.put(res, np.ones_like(ones), ones)
    
    
    return res

In [17]:
# ca male : (8.5+10.6+8.7+10.5+8.8+10.6+8.7+10.3+8.5+10.2+8.4+10.3)/2/6 
# ca female : (8.4+10.6+8.9+10.5+8.5+10.5+8.5+10.3+8.4+10.2+8.6+10.3)/2/6
#  plt : Platelets 
# TODO cpk should be checked
'''
Phosphate : 1

    "creati_1" : {
        "male": 1.0 ,
        "female": 0.85,
    },
    
'''
avg = {
    "cpk_1" : {
        "male":57.5,
        "female": 40,
    },
    "na_1" : 140,
    "k_1" : 4.75,
    "ca_1":{
        "male" : 9.5083,
        "female" : 9.475,
    } ,
    'ldh_1':219,
    "uricac_1":{
        "male":5.2,
        "female":4.2,
    },
    "plt_1":350000,
    "wbc_1":7.4e+3,
    
}

In [18]:
def replace_and_outPut(data, train = True, means = None, scalerAfterGender = None, scalerBeforeModels = None,
                      scaler_age = None, scaler_cret = None, model_age = None, model_cret = None,
                      model_final_nn = None, model_final_svc = None, model_final_rfc = None, model_final_for_avg = None, mid_nn = None, mid_nn_avg = None):
    
    
    if train :
        
        model_age_nn, model_cret_nn, model_final_nn, model_age_svr, model_cret_svr, model_final_svc, model_age_rfr, model_cret_rfr, model_final_rfc, model_final_for_avg, mid_nn, mid_nn_avg = get_models()
        model_age = model_age_nn
        model_cret = model_cret_nn
        means = []
    
    for i, iv in enumerate(data.T):
        if train :
            mean = np.nanmean(data.T[i,:])
            means.append(mean)
        else :
#             a = np.nanmean(data.T[i,:])
            mean = means[i]
        for j, jv in enumerate(iv):
            if str(jv) == str(np.nan):
                if i >=  len(ommit) :
                    continue 
                else :
                    data[j, i] = mean
    
                    
    df_new = pd.DataFrame(data, columns=df.columns)

    if gender_index is  not -1 :
        df_new["gender"] = df_new["gender"].apply(getGender)
    
    data = df_new.values
    
    
    if train :
        scalerAfterGender = preprocessing.StandardScaler().fit(data[:,:len(ommit)])
    data[:,:len(ommit)] = scalerAfterGender.transform(data[:,:len(ommit)])
    
    
    
    '''
    for the first columns we will use the staticly found means but for ohter cols we will use AI
    '''
    
    
    shuffled_and_replaced = replaceNan(data, num, df_new.columns[:num], avg, gender_index)
#     print(np.sum(np.isnan(shuffled_and_replaced), axis = 0), shuffled_and_replaced.shape[1],"first")

    if train :
        scalerBeforeModels = preprocessing.StandardScaler().fit(shuffled_and_replaced[:,:num])
    shuffled_and_replaced[:,:num] = scalerBeforeModels.transform(shuffled_and_replaced[:,:num])
    
    cret, scaler_age, scaler_cret, model_age, model_cret = replace_with_ml(shuffled_and_replaced, 
                                                            model_age, model_cret, "nn",
                                                           train = train, scaler_age = scaler_age,
                                                            scaler_cret = scaler_cret)
    
    
    
#     resFinal = np.zeros((5,2))
    
    X = cret[:, :-1]
    print(np.sum(np.isnan(cret), axis = 0), cret.shape[1])
    y = cret[:, -1]
    
    if (train):
        
        model_final_nn.fit(X, y, batch_size = 8 , epochs= 60, verbose = 0, class_weight={0:0.07,1:2.8}, shuffle = False)
        model_final_for_avg.fit(X, y, batch_size = 8 , epochs= 50, verbose = 0, shuffle = False, class_weight={0:0.1,1:2})

        model_final_svc.fit(X, y)

        model_final_rfc.fit(X, y)
        rfs.append(model_final_rfc.best_estimator_)

    y_pred_nn  = model_final_nn.predict(X) > 0.35
    y_pred_nn_for_avg = model_final_for_avg.predict(X) > 0.4
    y_pred_svc = model_final_svc.predict(X)
    y_pred_rfc = model_final_rfc.predict(X)
#     2 taa nn bayad injoori akhar bian
    y_pred_en  = get_res(y_pred_svc, y_pred_rfc, y_pred_nn, y_pred_nn_for_avg)
    
    if (not train) or True:
        print()
        print("nn", get_acc(y, y_pred_nn))
        print("nn_lower", get_acc(y, y_pred_nn_for_avg))
        print("svc", get_acc(y, y_pred_svc))
        print("rfc", get_acc(y, y_pred_rfc))
        print("en", get_acc(y, y_pred_en))
        print()

    resFinal = np.array(
        [list(get_acc(y, y_pred_nn)),
         list(get_acc(y, y_pred_nn_for_avg)),
        list(get_acc(y, y_pred_svc)),
        list(get_acc(y, y_pred_rfc)),
        list(get_acc(y, y_pred_en)),
        ]

    )
    
    all1 = (mid_nn.predict(X), model_final_nn.predict(X))
    all2 = (mid_nn_avg.predict(X), model_final_for_avg.predict(X))
    
    return resFinal, means, scalerAfterGender, scalerBeforeModels, scaler_age, scaler_cret, model_age, model_cret, model_final_nn, model_final_svc, model_final_rfc, model_final_for_avg, mid_nn, mid_nn_avg, X, y,all1, all2
    

In [19]:
def to_csvs(res, i, name="_Train_"):
    high_mid = pd.DataFrame(res[-2][0])
    hight_last = pd.DataFrame(res[-2][1])

    normal_mid = pd.DataFrame(res[-1][0])
    normal_final = pd.DataFrame(res[-1][1])

    Xpd = pd.DataFrame(res[-4])
    ypd = pd.DataFrame(res[-3])

    
    high_mid.to_csv(address_corr+"high_mid_ATF_"+name+str(i)+"_.csv", index=False)
    hight_last.to_csv(address_corr+"hight_last_ATF_"+name+str(i)+"_.csv", index=False)
    normal_mid.to_csv(address_corr+"normal_mid_ATF_"+name+str(i)+"_.csv", index=False)
    normal_final.to_csv(address_corr+"normal_final_ATF_"+name+str(i)+"_.csv", index=False)
    Xpd.to_csv(address_corr+"Xpd_ATF_"+name+str(i)+"_.csv", index=False)
    ypd.to_csv(address_corr+"ypd_ATF_"+name+str(i)+"_.csv", index=False)
    
    

In [20]:
def train_and_test(df, k = 10, matrix = None):
    
    if matrix is None :
        matrix = df.values

        np.random.seed(42)
        ind = np.arange(matrix.shape[0])
        np.random.shuffle(ind)
        matrix = matrix[ind]
    
    for i, vi in enumerate(matrix):
        for j, vj in enumerate(vi):
            if str(vj).strip() == "":
                if (j == len(cols)-1):
                    matrix[i, j] = 0
                else :
                    matrix[i, j] = np.nan
                
    matrix = matrix.astype(np.float32)
    dfN = pd.DataFrame(matrix, columns=df.columns)
    if gender_index is not -1 :
        matrix = dfN[dfN["gender"].isnull() ^ 1].values
    
    
    size = math.ceil(matrix.shape[0] / k)
    
    resTrains = []
    resTests  = []
    
    dictInfo = dict()
    
    for i in range(k):
        
        print("fold of k : ", i)
        
        start = i * size
        end   = (i + 1) * size
        
        if end > matrix.shape[0] :
            
            end = matrix.shape[0]
        
        train_data = np.concatenate([matrix[:start], matrix[end:]], axis = 0)
        test_data  = matrix[start:end]
        
        
        print("Train : ")
        res = replace_and_outPut(train_data.copy())
        
        to_csvs(res, i)
        resTrains.append(res[0])
        
        dictInfo[i] = {
            "model_age" : res[6],
            "model_cret" : res[7],
            "scaler_age" : res[4],
            "scaler_cret" : res[5],
        }
        
        print("Tests : ")
        res = replace_and_outPut(test_data.copy(), False, res[1], 
                                res[2], res[3], res[4], res[5], res[6], res[7], res[8],
                                res[9], res[10], res[11], res[12], res[13])
        resTests.append(res[0])
        to_csvs(res, i, "_Test_")

        
        print()
        
#         time.sleep(180)
        
    return resTrains, resTests, dictInfo

    
    

In [21]:
def shuffle_and_train_and_test(offset=0):
#     if you gave offset you need to run the cell for cols on the top one more time
    np.random.seed(42)
    
    
    global ommit
    global otherCpks
    global cols
    global df1
    global df
    global num
    global cret_index
    global gender_index
    global age_index
    
    get_global_vars(True)    
    
    cols1 = cols
    for i in range(offset, len(cols1)-1):
        print(cols1[i],":",i)
        del df
        df=df1[cols1[:i]+cols1[i+1:]]
        cols = list(df.columns)
        num = len(cols) - len([ 'age', 'gender','crat3f', "atn" ])
        if i >= cols1.index("age"):
            num += 1
        try :
            cret_index = cols.index('crat3f')
        except Exception as e :
            cret_index = -1
        try :
            gender_index = cols.index('gender')
        except Exception as e :
            gender_index = -1
        try :
            age_index= cols.index('age')
        except Exception as e :
            age_index = -1
        
        x = train_and_test(df)
        
        file1 = open("save_rfs_ARF_train_Copy"+str(i)+"_.pkl", mode="wb")
        file2 = open("save_rfs_ARF_test_Copy"+str(i)+"_.pkl", mode="wb")
        tr = np.array(x[0])
        ts = np.array(x[1])
        pickle.dump(tr, file1)
        pickle.dump(ts, file2)
        
        del file1
        del file2
        del tr
        del ts
        
        
#         time.sleep(60)
    print("final  : ")
    del df
    df=df1[cols1]
    cols = list(df.columns)
    num = len(cols) - len([ 'age', 'gender','crat3f', "atn" ])
    try :
        cret_index = cols.index('crat3f')
    except Exception as e :
        cret_index = -1
    try :
        gendear_index = cols.index('gender')
    except Exception as e :
        gender_index = -1
    try :
        age_index= cols.index('age')
    except Exception as e :
        age_index = -1
    x = train_and_test(df)
    file1 = open("Final_Copy_train_"+"_.pkl", mode="wb")
    file2 = open("Final_Copy_test_"+"_.pkl", mode="wb")
    tr = np.array(x[0])
    ts = np.array(x[1])
    pickle.dump(tr, file1)
    pickle.dump(ts, file2)
    return x
        
    
    
    

In [44]:
r = shuffle_and_train_and_test(14)

final  : 
fold of k :  0
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9276094276094277, 1.0)
nn_lower (0.9713804713804713, 0.9634146341463414)
svc (0.9974747474747475, 0.8902439024390244)
rfc (1.0, 1.0)
en (0.9713804713804713, 0.9634146341463414)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9621212121212122, 1.0)
nn_lower (1.0, 0.9)
svc (1.0, 0.9)
rfc (1.0, 0.9)
en (1.0, 0.9)


fold of k :  1
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [24]:
get_global_vars(True)

Index(['creati_1', 'sgot_1', 'phosph_1', 'cpk_1', 'ldh_1', 'wbc_1', 'plt_1',
       'uricac_1', 'na_1', 'k_1', 'ca_1', 'age', 'gender', 'crat3f', 'atn'],
      dtype='object')


In [46]:
r = train_and_test(df, k = 10)

fold of k :  0
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9276094276094277, 1.0)
nn_lower (0.9713804713804713, 0.9634146341463414)
svc (0.9974747474747475, 0.8902439024390244)
rfc (1.0, 1.0)
en (0.9713804713804713, 0.9634146341463414)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9621212121212122, 1.0)
nn_lower (1.0, 0.9)
svc (1.0, 0.9)
rfc (1.0, 0.9)
en (1.0, 0.9)


fold of k :  1
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9249578414839797, 1.0)
nn_lower (0.96964586846543, 0.9523809523809523)
svc (0.9983136593591906, 0.8809523809523809)
rfc (1.0, 1.0)
en (0.9704890387858347, 0.9523809523809523)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9328358208955224, 1.0)
nn_lower (0.9626865671641791, 1.0)
svc (1.0, 0.875)
rfc (1.0, 0.875)
en (0.9701492537313433, 1.0)


fold of k :  2
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


...


nn (0.9156829679595279, 1.0)
nn_lower (0.9688026981450253, 0.9523809523809523)
svc (0.9991568296795953, 0.8928571428571429)
rfc (1.0, 1.0)
en (0.9713322091062394, 0.9523809523809523)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9402985074626866, 1.0)
nn_lower (0.9701492537313433, 1.0)
svc (1.0, 1.0)
rfc (0.9925373134328358, 1.0)
en (0.9776119402985075, 1.0)


fold of k :  8
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9250841750841751, 1.0)
nn_lower (0.9595959595959596, 0.9518072289156626)
svc (0.9974747474747475, 0.891566265060241)
rfc (1.0, 1.0)
en (0.9638047138047138, 0.9518072289156626)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.8712121212121212, 1.0)
nn_lower (0.9393939393939394, 1.0)
svc (1.0, 0.8888888888888888)
rfc (0.9924242424242424, 1.0)
en (0.946969696969697, 1.0)


fold of k :  9
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9210747271200672, 1.0)
nn_lower (0.9739714525608733, 0.9529411764705882)
svc (0.998320738874895, 0.8941176470588236)
rfc (1.0, 1.0)
en (0.9739714525608733, 0.9529411764705882)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9302325581395349, 0.8571428571428571)
nn_lower (0.9922480620155039, 0.8571428571428571)
svc (1.0, 0.7142857142857143)
rfc (1.0, 0.8571428571428571)
en (0.9922480620155039, 0.8571428571428571)




In [47]:
np.mean(np.array(r[0]), axis = 0)

array([[0.93450341, 0.99878049],
       [0.97213547, 0.96148627],
       [0.99848435, 0.89764303],
       [1.        , 1.        ],
       [0.97432425, 0.96148627]])

In [48]:
np.mean(np.array(r[1]), axis = 0)

array([[0.93118405, 0.94123876],
       [0.96595651, 0.93123876],
       [0.99696252, 0.82129065],
       [0.99249522, 0.90335415],
       [0.96972756, 0.93123876]])

In [25]:
r = train_and_test(df, k = 10)

fold of k :  0
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9428090832632464, 1.0)
nn_lower (0.975609756097561, 0.9629629629629629)
svc (0.9974768713204374, 0.9012345679012346)
rfc (1.0, 1.0)
en (0.9781328847771237, 0.9629629629629629)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9545454545454546, 1.0)
nn_lower (0.9924242424242424, 0.9)
svc (1.0, 0.9)
rfc (1.0, 0.9)
en (0.9924242424242424, 0.9)


fold of k :  1
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9359730412805392, 1.0)
nn_lower (0.9696714406065712, 0.9518072289156626)
svc (0.9983150800336984, 0.891566265060241)
rfc (1.0, 1.0)
en (0.9721988205560236, 0.9518072289156626)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9477611940298507, 1.0)
nn_lower (0.9701492537313433, 1.0)
svc (1.0, 0.875)
rfc (1.0, 1.0)
en (0.9701492537313433, 1.0)


fold of k :  2
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


...


nn (0.9241786015164279, 1.0)
nn_lower (0.974726200505476, 0.963855421686747)
svc (0.9991575400168492, 0.9036144578313253)
rfc (1.0, 1.0)
en (0.9764111204717776, 0.963855421686747)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9552238805970149, 1.0)
nn_lower (0.9776119402985075, 1.0)
svc (1.0, 1.0)
rfc (1.0, 1.0)
en (0.9850746268656716, 1.0)


fold of k :  8
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9411269974768713, 1.0)
nn_lower (0.9663582842724979, 0.9634146341463414)
svc (0.9966358284272497, 0.9024390243902439)
rfc (1.0, 1.0)
en (0.9722455845248108, 0.9634146341463414)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9015151515151515, 0.8888888888888888)
nn_lower (0.9393939393939394, 1.0)
svc (1.0, 0.8888888888888888)
rfc (0.9924242424242424, 1.0)
en (0.946969696969697, 0.8888888888888888)


fold of k :  9
Train : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15


c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\programdata\anaconda3\envs\myonlyenv6\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)



nn (0.9437919463087249, 1.0)
nn_lower (0.9790268456375839, 0.9642857142857143)
svc (0.9983221476510067, 0.9047619047619048)
rfc (0.9991610738255033, 1.0)
en (0.9798657718120806, 0.9642857142857143)

Tests : 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 15

nn (0.9457364341085271, 0.8571428571428571)
nn_lower (0.9844961240310077, 0.8571428571428571)
svc (1.0, 0.8571428571428571)
rfc (1.0, 0.7142857142857143)
en (0.9844961240310077, 0.8571428571428571)




In [26]:
np.mean(np.array(r[0]), axis = 0)

array([[0.94616234, 0.99876543],
       [0.97560544, 0.96713327],
       [0.99840152, 0.90492575],
       [0.99991611, 1.        ],
       [0.97829707, 0.96713327]])

In [27]:
np.mean(np.array(r[1]), axis = 0)

array([[0.94177768, 0.93717893],
       [0.96521126, 0.93829004],
       [0.99696858, 0.8413456 ],
       [0.99398284, 0.90733766],
       [0.96971241, 0.92717893]])